In [217]:

from qiskit.circuit import Gate

In [218]:
import qiskit
from qiskit.providers.aer.pulse import PulseSystemModel
from qutip import sigmax
from qutip.control.grape import _overlap
# from notebooks.qiskit_scripts import pulse_to_gate_1q
from notebooks.qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
%matplotlib inline
# import matplotlib.pyplot as plt
import numpy as np
# import datetime
# from qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
from notebooks.qiskit_scripts import *
# from qutip import identity, sigmax, sigmaz, sigmay, hadamard_transform

from qiskit.tools.qi.qi import outer

In [219]:
from qiskit import IBMQ, pulse, ClassicalRegister, QuantumRegister
from qiskit.circuit import QuantumCircuit
from qiskit.pulse import Play

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')
armonk_backend = backend

ibmqfactory.load_account:WARNING:2020-07-29 16:51:00,303: Credentials are already in use. The existing account in the session will be replaced.


In [220]:
freq_est = 4974450442.586346
omegad0 = 31919806.545849085
dt = armonk_backend.configuration().dt
wq0 = 2 * np.pi * freq_est

scale_factor = 1e4
n_ts = int(1600)
evo_time = dt * n_ts * scale_factor
delta=wq0/16
# delta=0
# default_delta = wq0/16
delta_freq = delta / 2 / np.pi
phase=np.pi/3
optimized_pulse_seq, qutip_result = qutip_optimize_wrapper(delta/scale_factor, omegad0/scale_factor, evo_time, n_ts, final_evo=True, target='sigmax', p_type='RND')

Final evolution
Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = False
Qobj data =
[[-2.52544713e-06-2.92248508e-06j  9.90742286e-01-1.35756113e-01j]
 [ 9.90742029e-01-1.35757989e-01j  1.64620941e-06-3.49410888e-06j]]



In [ ]:
delta

In [222]:
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['omegad0'] = omegad0
getattr(armonk_backend.configuration(), 'hamiltonian')['vars']['wq0'] = wq0


armonk_backend.configuration().qubit_lo_freq=[freq_est - delta_freq]

In [223]:
gate, inst_map = pulse_to_gate_1q(backend, pulse.SamplePulse(optimized_pulse_seq))
pulse_aer_backend = qiskit.providers.aer.PulseSimulator(
    configuration=armonk_backend.configuration())
armonk_model = PulseSystemModel.from_backend(armonk_backend)

/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/pulse/pulse_lib/sample_pulse.py:149: DeprecationWarning: Calling `SamplePulse` with a channel is deprecated. Instantiate the new `Play` instruction directly with a pulse and a channel. In this case, please use: `Play(SamplePulse(samples), DriveChannel(0))`.
  warnings.warn("Calling `{}` with a channel is deprecated. Instantiate the new `Play` "
/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/pulse/pulse_lib/pulse.py:46: DeprecationWarning: Calling `SamplePulse` with a channel is deprecated. Instantiate the new `Play` instruction directly with a pulse and a channel. In this case, please use: `Play(SamplePulse(array([-1.+0.j, -1.+0.j, -1.+0.j, ..., -1.+0.j, -1.+0.j, -1.+0.j])), DriveChannel(0))`.
  warnings.warn("Calling `{}` with a channel is deprecated. Instantiate the new `Play` "
/Users/benrosand/anaconda3/envs/qutip/lib/python3.8/site-packages/qiskit/providers/models/pulsedefaults.py:28

In [224]:
#NOTE we removed measurement for qpt?
q = QuantumRegister(1)
c = ClassicalRegister(1)
my_circ = qiskit.QuantumCircuit(q, c)
my_circ.append(gate,q)
# my_circ.x(q)
my_circ.measure([0], [0])

small_transp_circ = qiskit.transpile(my_circ, backend=armonk_backend, basis_gates=armonk_backend.configuration().basis_gates + ['opt_gate'])
small_gate_sched = qiskit.schedule(small_transp_circ, inst_map=inst_map, backend=pulse_aer_backend, meas_map=[[0]])

program = assemble(small_gate_sched, backend=armonk_backend, meas_level=2, meas_return='single',
                   shots=4096, qubit_lo_freq=[freq_est-delta_freq])
#
job = qiskit.execute(small_gate_sched, pulse_aer_backend, shots=4096, system_model=armonk_model)
# prog_job = qiskit.execute(program, pulse_aer_backend, shots=4096, system_model=armonk_model)
prog_job = pulse_aer_backend.run(program,armonk_model)

In [225]:
# job.result().get_counts()
prog_job.result().get_counts()

{'0': 1917, '1': 2179}